In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import os
import seaborn as sns
import umap
import csv
import warnings
from collections import OrderedDict
from collections import defaultdict
from scipy import signal, stats
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import axes3d
from matplotlib.patches import Patch
from sklearn.preprocessing import StandardScaler
from pydmd import MrDMD, DMD, OptDMD

plt.rcParams.update({'figure.max_open_warning': 0})
warnings.simplefilter("ignore")

%run /media/turritopsis/katie/grooming/t1-grooming/grooming_functions.ipynb
%matplotlib inline

In [ ]:
# delay embedding
def time_embed(points, stacks=8, stride=1, ixs=None):
    if ixs is None:
        ixs = np.arange(len(points)-stacks*stride-1)
    if len(points.shape) == 1:
        ps = np.vstack([points[ixs+i*stride] for i in range(stacks)]).T
    else:
        ps = np.hstack([points[ixs+i*stride] for i in range(stacks)])
    return ps

In [ ]:
# load data
def adjust_rot_angles(angles, angle_names):
    conds = ['2', '3', 'L1A', 'L1B', 'L1C', 'R1A', 'R1B', 'R1C']
    offsets = np.array([-50, -20, 20, -70, 10, 20, 70, -30])
    for j in range(len(conds)):
        rot_angs = [r for r in angle_names if '_rot' in r and conds[j] in r]
        for ang in rot_angs:
            r = np.array(angles[ang])
            r[r > offsets[j]] = r[r > offsets[j]] - 360
            angles[ang] = r
        
    abduct_angs = [r for r in angle_names if '_abduct' in r or 'A_flex' in r]
    for ang in abduct_angs:
        r = np.array(angles[ang])
        r[r > 50] = r[r > 50] - 360
        angles[ang] = r
        
    return angles

prefix = r'/media/turritopsis/pierre/gdrive/latest/behavior/T1_grooming'
data_path = os.path.join(prefix, 'T1_grooming_all.csv.gz')
data = pd.read_csv(data_path, compression = 'gzip')
data['behavior_bout'] = data['T1_grooming_bout_number']
data['flyid'] = data['fly'].astype(str) + ' ' + data['date'].astype(str)
data = data[~data.behavior_bout.isnull()]
angle_names = [v for v in data.columns
              if some_contains(v, ['_flex', '_abduct', '_rot'])
             and v[:2] in ['L1', 'R1']]

data = adjust_bout_numbers(data)
bout_length_dict = get_bout_lengths(data)
data = remove_short_bouts(data, 60)
data = correct_angles(data, angle_names)
data = adjust_rot_angles(data, angle_names)

###### dmd on a single grooming bout 

In [ ]:
bout = 60
check = data['behavior_bout'] == bout
d = data.loc[check]
angle_vars = [v for v in data.columns
              if some_contains(v, ['_flex', '_abduct', '_rot'])
             and v[:2] in ['L1', 'R1']]
X = np.array(d.loc[:, angle_vars])
X_scaled = StandardScaler().fit_transform(X)
X_scaled = time_embed(X_scaled, stacks=10, stride=1)
dmd = DMD(svd_rank=10, tlsq_rank=0, exact=True, opt=True)
# dmd = MrDMD(svd_rank=3, tlsq_rank=0, max_level=8, max_cycles=2)
dmd.fit(X_scaled.T)
X_recon = dmd.reconstructed_data.T

In [ ]:
cols = X.shape[1]
cmap = plt.get_cmap('tab20')

fig = plt.figure()
plt.title('input angles', fontsize = 14)
plt.xlabel('time (frames)', fontsize = 14)
plt.ylabel('angle (deg)', fontsize = 14)
for j in range(cols):
    plt.plot(X[:, j])

fig = plt.figure()
plt.title('input angles (normalized)', fontsize = 14)
plt.xlabel('time (frames)', fontsize = 14)
plt.ylabel('amplitude (norm.)', fontsize = 14)
for j in range(cols):
    plt.plot(X_scaled[:, j])...

In [ ]:
dmd.plot_eigs(show_axes=True, show_unit_circle=True, figsize=(6, 6))
dmd.eigs

###### signal reconstruction

In [ ]:
for j in range(X.shape[1]):
    fig, ax = plt.subplots(1, 2, sharex=True, sharey=True, figsize = (14, 3))
    ax = ax.T.flatten()
    ax[0].plot(X_scaled[:, j], 'xkcd:indigo blue')
    ax[0].set_xlabel('time (frames)', fontsize = 14)
    ax[0].set_ylabel('amplitude (norm.)', fontsize = 14)
    ax[0].set_title('{} angles (original)'.format(angle_vars[j]), fontsize = 14)
    
    ax[1].plot(X_recon[:, j], 'xkcd:wine red')
    ax[1].set_xlabel('time (frames)', fontsize = 14)
    ax[1].set_title('{} angles (reconstruction)'.format(angle_vars[j]), fontsize = 14)
    ax[1].set_ylabel('amplitude (norm.)', fontsize = 14)
    ax[1].tick_params(labelleft = True)
    
    plt.subplots_adjust(wspace = 0.2)
    plt.show()

In [ ]:
dynamics = dmd.dynamics
fig = plt.figure()
plt.title('time evolution of dynamics', fontsize = 14)
plt.xlabel('time', fontsize = 14)
plt.ylabel('amplitude', fontsize = 14)
plt.plot(range(len(dynamics.T)), dynamics.T)

for j in range(dynamics.shape[0]):
    fig = plt.figure()
    plt.title('n = ' + str(j+1), fontsize = 14)
    plt.xlabel('time', fontsize = 14)
    plt.ylabel('amplitude', fontsize = 14)
    plt.plot(range(len(dynamics.T[:, j])), dynamics.T[:, j])
    plt.show()

In [ ]:
modes = dmd.modes
fig = plt.figure(figsize = (12, 3))
plt.title('DMD modes', fontsize = 14)
plt.plot(modes.T)
plt.show()

for j in range(modes.shape[0]):
    fig = plt.figure(figsize = (12, 3))
    plt.title('DMD modes', fontsize = 14)
    plt.plot(modes.T[:, j])
    plt.show()

In [ ]:
x = dmd.plot_modes_2D(index_mode = 1, x = np.arange(dmd.snapshots.shape[0]), y = np.arange(dmd.snapshots.shape[1]))

###### signal reconstruction (specific to mrDMD)

In [ ]:
pdyna = dmd.partial_dynamics(level=3)
print('The number of modes in the level number 1 is {}'.format(pdyna.shape[0]))
fig = plt.plot(range(len(pdyna.real.T)), pdyna.real.T)

In [ ]:
pdata = dmd.partial_reconstructed_data(level=4)
plt.plot(pdata.T, X)

###### dmd on multiple grooming bouts

In [ ]:
def get_all_bouts(data, angle_vars, delay_embed = False, stacks = None, stride = None):
    
    all_X = []
    all_Y = []
    bout_numbers = np.unique(data.behavior_bout)
    for j in range(len(bout_numbers)): 
        bout = fly_data[data.behavior_bout == bout_numbers[j]]
        X = np.array(bout.loc[:, angle_vars])
        X_scaled = StandardScaler().fit_transform(X)
        if delay_embed:
            X_scaled = time_embed(X_scaled, stacks=stacks, stride=stride)
        all_X.extend(X_scaled[:-1])
        all_Y.extend(X_scaled[1:])

    all_X = np.array(all_X).T
    all_Y = np.array(all_Y).T

    return all_X, all_Y

angle_vars = [v for v in data.columns
              if some_contains(v, ['_flex', '_abduct', '_rot'])
             and v[:2] in ['L1', 'R1']]
fly_name = '4_0 5222019'
fly_data = data[data.flyid == fly_name]
all_X, all_Y = get_all_bouts(fly_data, angle_vars)
# all_X, all_Y = get_all_bouts(fly_data, angle_vars, delay_embed = True, stacks = 10, stride = 10)

dmd = OptDMD(svd_rank=0, tlsq_rank=0, exact=True, opt=True)
dmd.fit(all_X, all_Y)
# X_recon = dmd.reconstructed_data.T
# dmd.predict(all_X[:, 300:])

In [ ]:
# fit dmd on all bouts for a fly except 1, then pass in the 
# first half of a bout and have it predict the second half
angle_vars = [v for v in data.columns
              if some_contains(v, ['_flex', '_abduct', '_rot'])
             and v[:2] in ['L1', 'R1']]
fly_name = '4_0 5222019'
fly_data = data[data.flyid == fly_name]
delay_embed = False
stacks = 5
stride = 15

all_X = []
all_Y = []
bout_numbers = np.unique(fly_data.behavior_bout)
for j in range(len(bout_numbers)-1): 
    bout = fly_data[fly_data.behavior_bout == bout_numbers[j]]
    X = np.array(bout.loc[:, angle_vars])
    X_scaled = StandardScaler().fit_transform(X)
    if delay_embed:
        X_scaled = time_embed(X_scaled, stacks=stacks, stride=stride)
    all_X.extend(X_scaled[:-1])
    all_Y.extend(X_scaled[1:])

all_X = np.array(all_X).T
all_Y = np.array(all_Y).T
last_bout =  fly_data[data.behavior_bout == bout_numbers[-1]]
bout_X = np.array(last_bout.loc[:, angle_vars]).T
bout_X = StandardScaler().fit_transform(bout_X)
if delay_embed:
    bout_X = time_embed(bout_X, stacks=stacks, stride=stride)

dmd = OptDMD(svd_rank=0, tlsq_rank=0, exact=True, opt=True)
dmd.fit(all_X, all_Y)
half_bout = bout_X[:, :int(bout_X.shape[1]/2)]
predicted_bout = dmd.predict(half_bout)

In [ ]:
fig = plt.figure(figsize = (15,3))
plt.plot(bout_X.T - bout_X.T[0, :])
plt.show()

Y = dmd.predict(all_X[:, :300])
fig = plt.figure(figsize = (15, 3))
plt.plot(bout_X[:, :int(bout_X.shape[1]/2)].T)
plt.show()


fig = plt.figure(figsize = (15, 3))
plt.plot(Y.T)
plt.show()


In [ ]:
for j in range(X.shape[1]):
    fig, ax = plt.subplots(1, 2, sharex=True, sharey=True, figsize = (14, 3))
    ax = ax.T.flatten()
    ax[0].plot(X_scaled[:, j], 'xkcd:indigo blue')
    ax[0].set_xlabel('time (frames)', fontsize = 14)
    ax[0].set_ylabel('amplitude (norm.)', fontsize = 14)
    ax[0].set_title('{} angles (original)'.format(angle_vars[j]), fontsize = 14)
    
    ax[1].plot(X_recon[:, j], 'xkcd:wine red')
    ax[1].set_xlabel('time (frames)', fontsize = 14)
    ax[1].set_title('{} angles (reconstruction)'.format(angle_vars[j]), fontsize = 14)
    ax[1].set_ylabel('amplitude (norm.)', fontsize = 14)
    ax[1].tick_params(labelleft = True)
    
    plt.subplots_adjust(wspace = 0.2)
    plt.show()